In [93]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest,chi2
import seaborn as sns

In [94]:
df =sns.load_dataset('tips')

In [95]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [96]:
df.drop(columns=['day','time'],inplace=True)

In [97]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop(['tip'],axis=True),df['tip'],test_size=0.2,random_state=42)

In [98]:
x_train.dtypes

total_bill     float64
sex           category
smoker        category
size             int64
dtype: object

In [99]:
x_train.shape,x_test.shape

((195, 4), (49, 4))

In [100]:
from sklearn.compose import ColumnTransformer

In [101]:
x_train

,total_bill,sex,smoker,size
228,13.28,Male,No,2
208,24.27,Male,Yes,2
96,27.28,Male,Yes,2
167,31.71,Male,No,4
84,15.98,Male,No,2
...,...,...,...,...
106,20.49,Male,Yes,2
14,14.83,Female,No,2
92,5.75,Female,Yes,2
179,34.63,Male,Yes,2


In [115]:
level1 = ColumnTransformer([
    ('totalbill',SimpleImputer(),[0,3]),
    ('ys',SimpleImputer(strategy='most_frequent'),[2]),
    ('x',SimpleImputer(strategy='median'),[4])
])

In [116]:
level2= ColumnTransformer([
    ('sex',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1]),
    ('smoker',OrdinalEncoder(categories=[['No','Yes']]),[2])
   
])

In [117]:
level2.fit(x_train,y_train)

,transformers,"[('sex', ...), ('smoker', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,None
,sparse_output,False


In [118]:
level3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [119]:
level4 =SelectKBest(score_func=chi2,k=6)

In [120]:
level5 = DecisionTreeClassifier()

In [121]:
pipe = Pipeline([
    ('level1', level1),
    ('level2', level2),
    ('level3', level3),
    ('level4', level4),
    ('level5', level5)
])


In [122]:
pipelinemake = make_pipeline(level1,level2,level3,level4,level5)

In [123]:
pipelinemake.fit(x_train,y_train)

ValueError: all features must be in [0, 3] or [-4, 0]

In [124]:
y_pred = pipelinemake.predict(x_test)

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [125]:
accuracy_score(y_test,y_pred)

NameError: name 'y_pred' is not defined

In [126]:
pipe.fit(x_train,y_train)

ValueError: all features must be in [0, 3] or [-4, 0]

In [127]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load dataset
tips = sns.load_dataset("tips")

# Create binary target
tips['high_tip'] = (tips['tip'] > tips['tip'].median()).astype(int)

# Features and target
X = tips[['total_bill','size','sex','smoker','day','time']]
y = tips['high_tip']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing
numeric_features = ['total_bill','size']
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

categorical_features = ['sex','smoker','day','time']
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Full pipeline
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_select', SelectKBest(score_func=chi2, k=6)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit
pipe.fit(X_train, y_train)

# Predict
y_pred = pipe.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.6530612244897959


In [128]:
# Fit
pipe.fit(X_train, y_train)


,steps,"[('preprocessor', ...), ('feature_select', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [129]:
x_train

,total_bill,sex,smoker,size
228,13.28,Male,No,2
208,24.27,Male,Yes,2
96,27.28,Male,Yes,2
167,31.71,Male,No,4
84,15.98,Male,No,2
...,...,...,...,...
106,20.49,Male,Yes,2
14,14.83,Female,No,2
92,5.75,Female,Yes,2
179,34.63,Male,Yes,2


In [130]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

# Column transformers
level1 = ColumnTransformer([
    ('totalbill', SimpleImputer(), [0,3]),
    ('ys', SimpleImputer(strategy='most_frequent'), [2]),
    ('x', SimpleImputer(strategy='median'), [3])   # you forgot a column index here
])

level2 = ColumnTransformer([
    ('sex', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1]),
    ('smoker', OrdinalEncoder(categories=[['No','Yes']]), [2])
])

level3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

level4 = SelectKBest(score_func=chi2, k=6)
level5 = DecisionTreeClassifier()

# Pipeline definition
pipe4 = Pipeline([
    ('level1', level1),
    ('level2', level2),
    ('level3', level3),
    ('level4', level4),
    ('level5', level5)
])

# Fit AFTER pipeline is defined
pipe4.fit(x_train, y_train)


,steps,"[('level1', ...), ('level2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('totalbill', ...), ('ys', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [131]:
y_p = pipe4.predict(x_test)

In [135]:
accuracy_score(y_test,y_p)

0.6938775510204082